# Hypergeometric enrichment analysis
The goal of this code is to replicate the enrichment analyses done in [Plaisier, et al. 2009](https://doi.org/10.1371/journal.pgen.1000642). There are five different analyses that we will attempt to replicate:

1. Link USF1 over-expression (OE) with rs3737787 correlated genes
2. Link USF1 direct target genes (Chip-ChIP) with rs3737787 correlated genes
3. Link rs3737787 correlated genes with FCHL differential gene expression (DEGs)
4. Link co-expression modules to FCHL DEGs
5. Link co-expression modules to rs3737787 correlated genes

Our analysis will be a little modified, but essentially the same.

### First we need to install packages that are not installed:

In [2]:
## Functional enrichment analysis
# pip install mygene
# pip install gseapy

### Next load up packages needed for analyses:
Please be sure to install what is missing if they give an error.

In [4]:
import pandas as pd
import GEOparse as geo
from scipy.stats import hypergeom
from statsmodels.stats import multitest
import gseapy as gp
import mygene
mg = mygene.MyGeneInfo()

### Load up [GSE17170](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE17170) which is the GEO ID for this study
We need to get the background set of genes (M) for this study. This will be accomplished by loading the data in GEOparse and converting the Affymetrix probe IDs into Entrez IDs.

In [6]:
fchl = geo.get_GEO(filepath="data/GSE17170_family.soft.gz")
gexpFCHL = pd.concat([fchl.gsms[gsm].table['VALUE'] for gsm in fchl.gsms.keys()],axis=1)
gexpFCHL.index = list(fchl.gsms[list(fchl.gsms.keys())[0]].table['ID_REF'])
gexpFCHL.columns = fchl.gsms.keys()
entrez = fchl.gpls['GPL570'].table.loc[fchl.gpls['GPL570'].table['ID'].isin(gexpFCHL.index),'ENTREZ_GENE_ID']
allGenes = set(entrez.dropna())

31-Oct-2024 07:59:23 INFO GEOparse - Parsing data/GSE17170_family.soft.gz: 
31-Oct-2024 07:59:23 DEBUG GEOparse - DATABASE: GeoMiame
31-Oct-2024 07:59:23 DEBUG GEOparse - SERIES: GSE17170
31-Oct-2024 07:59:23 DEBUG GEOparse - PLATFORM: GPL570
C:\Users\cplaisie\AppData\Local\anaconda3\Lib\site-packages\GEOparse\GEOparse.py:401: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  return read_csv(StringIO(data), index_col=None, sep="\t")
31-Oct-2024 07:59:25 DEBUG GEOparse - SAMPLE: GSM429486
31-Oct-2024 07:59:26 DEBUG GEOparse - SAMPLE: GSM429487
31-Oct-2024 07:59:26 DEBUG GEOparse - SAMPLE: GSM429488
31-Oct-2024 07:59:26 DEBUG GEOparse - SAMPLE: GSM429489
31-Oct-2024 07:59:26 DEBUG GEOparse - SAMPLE: GSM429490
31-Oct-2024 07:59:26 DEBUG GEOparse - SAMPLE: GSM429491
31-Oct-2024 07:59:26 DEBUG GEOparse - SAMPLE: GSM429492
31-Oct-2024 07:59:26 DEBUG GEOparse - SAMPLE: GSM429493
31-Oct-2024 07:59:26 DEBUG GEOparse - SAMPLE: GSM429494
31-Oct-

In [7]:
print(list(allGenes)[0:100])

['25923', '79086', '80223', '55862', '56647', '7174', '8705', '10924', '79663', '220', '7695', '6505', '79670', '64769', '80790', '10600', '3460', '5428', '8623', '50', '27106', '123207', '56674', '51747', '59269', '55869', '728264', '283537', '3223', '81887', '8943', '9738', '5226', '2029', '5423', '29087', '11161', '404093', '2319', '9766', '284613', '9470', '57175', '10561', '155061', '6275', '90826', '91612', '708', '63977', '2683', '54977', '8542', '59307', '11198', '7048', '80824', '10524', '23370', '8996', '57157', '100996412', '9188', '2202', '51150', '9802', '11145', '7041', '10180', '23136', '171568', '5376', '7071', '328', '54677', '1801 /// 124641', '79017', '4720', '3131', '222389', '64375', '2630', '83638', '207063', '57228', '161198', '374291', '28512', '2121', '5141', '140609', '8434', '399474', '3720', '124454', '50717', '84759', '90024', '8546', '8621']


There are some funky IDs with ///, so lets clean that up by taking splitting them and keeping all IDs. This section uses a very helpful tool called list comprehension:  https://realpython.com/list-comprehension-python/

We also employ a helpful trick made possible through list comprehension that allows us to un-nest nested lists:

`[item for sublist in list for item in sublist]`

In [9]:
allGenes = set([int(i3) for i2 in [i.split(' /// ') for i in allGenes] for i3 in i2])
print(list(allGenes)[0:100])

[102465536, 2, 13, 14, 16, 19, 20, 21, 22, 163859, 23, 25, 100859930, 27, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 163882, 131118, 47, 48, 50, 51, 52, 53, 54, 59, 60, 81, 86, 87, 90, 91, 92, 93, 94, 95, 97, 98, 100, 102, 103, 104, 108, 109, 112, 113, 115, 117, 118, 120, 123, 125, 126, 127, 128, 102465666, 102465667, 196740, 132, 133, 102465669, 142, 143, 150, 153, 154, 156, 157, 158, 159, 161, 162, 163, 164, 165, 166, 172, 175, 178, 182, 185, 187, 191, 199, 202, 203, 204, 205, 207, 208, 210]


Therefore M (background gene set) is:

In [11]:
print(len(allGenes))

9946


### Supplementary tables with DEGs are in PDFs
A lot of useful data can be found in supplementary tables, and you should remember to put useful information for your mansucript's into supplement!

#### *USF1 over-expression DEGs:*  https://doi.org/10.1371/journal.pgen.1000642.s003

In [13]:
# Load up USF1 over-expression (OE) differentially expressed genes (DEGs)
# https://doi.org/10.1371/journal.pgen.1000642.s003
df_USF1_OE_DEGs = pd.read_csv('data/df_USF1_OE_DEGs.csv', header=0, index_col=0) 
USF1_OE_DEGs = set(df_USF1_OE_DEGs['Entrez ID'].dropna()).intersection(allGenes)
print(len(USF1_OE_DEGs)) # 2140

2140


#### *USF1 target genes (from [TFBSDB](http://tfbsdb.systemsbiology.net)):*  http://tfbsdb.systemsbiology.net/tfgenes_csv/V_USF_02_M00122

In [15]:
# Load up USF1 target genes (from TFBS_DB)
# http://tfbsdb.systemsbiology.net/searchtf?searchterm=V_USF_02_M00122
# http://tfbsdb.systemsbiology.net/tfgenes_csv/V_USF_02_M00122
df_USF_M00122_target_genes = pd.read_csv('data/V_USF_02_M00122_genes.tsv', delimiter='\t', index_col=0, header=0)
USF_M00122_target_genes = set([i for i in df_USF_M00122_target_genes.index]).intersection(allGenes)
print(len(USF_M00122_target_genes)) # 319 (NEW source)

319


#### *rs3737787 correlated genes:*  https://doi.org/10.1371/journal.pgen.1000642.s004
We modified this to only look at postively associated genes, and those that had a p-value less than 0.01.

In [17]:
# Load up rs3737787 correlated genes
# https://doi.org/10.1371/journal.pgen.1000642.s004
df_rs3737787_corr = pd.read_csv('data/df_rs3737787_corr.csv', header=0, index_col=0)
df_rs3737787_corr_up = df_rs3737787_corr.loc[df_rs3737787_corr['Effect \nEstimate'].astype(float)>0]
df_rs3737787_corr_up_0_01 = df_rs3737787_corr_up.loc[df_rs3737787_corr_up['P-value'].astype(float)<0.01]
rs3737787_corr_up_0_01 = set(df_rs3737787_corr_up_0_01['Entrez ID'].dropna()).intersection(allGenes)
print(len(rs3737787_corr_up_0_01)) # 145 (very different than what is in the paper, because we are using extra filters)

145


#### *FCHL DEGs:*  https://doi.org/10.1371/journal.pgen.1000642.s005

In [19]:
# Load up USF1 over-expression (OE) differentially expressed genes (DEGs)
# https://doi.org/10.1371/journal.pgen.1000642.s005
df_FCHL_DEGs = pd.read_csv('data/df_FCHL_DEGs.csv', header=0, index_col=0)
FCHL_DEGs = set(df_FCHL_DEGs['Entrez ID'].dropna()).intersection(allGenes)
print(len(FCHL_DEGs)) # 2058

2058


### Hypergeometric enrichment analysis
Now that all the gene sets are available lets compute the overlaps.

1. Link USF1 over-expression (OE) with rs3737787 correlated genes
2. Link USF1 direct target genes (Chip-ChIP) with rs3737787 correlated genes
3. Link rs3737787 correlated genes with FCHL differential gene expression (DEGs)
4. Link co-expression modules to FCHL DEGs
5. Link co-expression modules to rs3737787 correlated genes

#### 1. USF1 OE DEGs vs. rs3737787 correlated/upregulated genes (cutoff 0.01)

In [21]:
k = len(USF1_OE_DEGs.intersection(rs3737787_corr_up_0_01))
M = len(allGenes)
n = len(USF1_OE_DEGs)
N = len(rs3737787_corr_up_0_01)
p_value = hypergeom.sf(k, M, n, N)
print([k,M,n,N,p_value])

[51, 9946, 2140, 145, 4.704198844055333e-05]


#### 2. USF1 target genes vs. rs3737787 correlated genes

In [23]:
k = len(USF_M00122_target_genes.intersection(rs3737787_corr_up_0_01))
M = len(allGenes)
n = len(USF_M00122_target_genes)
N = len(rs3737787_corr_up_0_01)
p_value = hypergeom.sf(k, M, n, N)
print([k,M,n,N,p_value])

[11, 9946, 319, 145, 0.002399271045278664]


#### 3. FCHL DEGs vs. rs3737787 correlated genes

In [25]:
k = len(FCHL_DEGs.intersection(rs3737787_corr_up_0_01))
M = len(allGenes)
n = len(FCHL_DEGs)
N = len(rs3737787_corr_up_0_01)
p_value = hypergeom.sf(k, M, n, N)
print([k,M,n,N,p_value])

[39, 9946, 2058, 145, 0.0278231553960585]


P-values for each of these analyses are less than 0.05, which means they show significant enrichment. The interpretation is:

- **1-2** linked USF1 regulation with rs3737787
- **3** linked rs3737787 with FCHL differential gene expression

## Comparisons with co-expression modules
> The gene expression variation between individuals in the Mexican FCHL fat biopsies is due to a combination of genetic and environmental factors. Ascertainment for disease status enriches the cases with factors leading to hyperlipidemia, and contrasting the cases with the controls will allow these factors to be more easily identified. We utilized weighted gene co-expression networking analysis (WGCNA) to identify gene co-expression modules summarizing the main trends in variation for the Mexican FCHL case/control fat biopsies. The WGCNA method clustered the 14,942 gene expression probes on the Mexican FCHL case/control microarrays into 28 gene co-expression modules ([Table S4](https://doi.org/10.1371/journal.pgen.1000642.s006)).

**Figure 1. Correlation of module eigengenes (ME) with clinical traits and rs3737787 SNP genotypes.** The rows are labeled by the ME color, and in parentheses is the number of genes in the module. The columns are labeled by SNP or clinical trait. The quantitative clinical traits, all except for FCHL which is qualitative, were corrected for significant covariates (age and/or sex) and standardized before use in analyses. The correlation coefficients are shown for each cell, and in parentheses is the p-value for the significance of the correlation. Cells are colorized based on the strength and sign of the correlation according to the scale on the right hand side of the figure. The tan module has been renamed to the USF1-regulated FCHL-associated (URFA) module.
![co-expression analysis](data/pgen.1000642.g001.png)
### Load up module memberships
Extracted only the Affymetrix probe ID to WGCNA module membership from Table S4. We will load that up now:

In [27]:
module_membership = pd.read_csv('data/moduleMembership_Plaisier2009.csv',header=0, index_col=0)
print(module_membership)

              WGCNA Module
Affy. Probes              
208685_x_at        Skyblue
208610_s_at        Skyblue
219437_s_at        Skyblue
207966_s_at        Skyblue
214911_s_at        Skyblue
...                    ...
204753_s_at           Grey
218315_s_at         Purple
219037_at     Midnightblue
235318_at      Lightyellow
208815_x_at           Blue

[14942 rows x 1 columns]


We want to turn that into something more usable for us. A dictionary of module to a list of probe IDs would work perfectly here. Let's use [dictionary comprehension](https://towardsdatascience.com/10-examples-to-master-python-dictionary-comprehensions-7aaa536f5960) to build the dictionary.

modules = `{modules:list(probe IDs), ...}`

In [29]:
modules = {i:list(module_membership.loc[module_membership['WGCNA Module']==i].index) for i in set(module_membership['WGCNA Module'])}
print(modules.keys())

dict_keys(['Cyan', 'Pink', 'Darkorange', 'Red', 'Salmon', 'Midnightblue', 'Darkred', 'Purple', 'Grey60', 'Skyblue', 'White', 'Greenyellow', 'Lightyellow', 'Orange', 'Royalblue', 'Lightgreen', 'Darkturquoise', 'Green', 'Darkgrey', 'Yellow', 'Lightcyan', 'Grey', 'Tan', 'Blue', 'Turquoise', 'Black', 'Magenta', 'Brown', 'Darkgreen'])


In [30]:
print(modules['Tan'][0:100])

['203359_s_at', '204971_at', '225670_at', '226195_at', '204218_at', '203517_at', '213902_at', '225330_at', '230174_at', '225593_at', '225684_at', '203327_at', '223100_s_at', '214733_s_at', '212226_s_at', '217922_at', '209268_at', '238890_at', '218138_at', '215947_s_at', '214719_at', '209694_at', '203545_at', '206833_s_at', '226196_s_at', '225480_at', '218447_at', '205201_at', '235509_at', '208633_s_at', '215000_s_at', '214033_at', '203042_at', '230257_s_at', '200713_s_at', '209208_at', '50374_at', '202348_s_at', '221511_x_at', '225108_at', '209934_s_at', '224376_s_at', '204839_at', '202544_at', '212371_at', '226671_at', '202088_at', '205055_at', '201899_s_at', '224180_x_at', '200736_s_at', '204079_at', '221255_s_at', '226851_at', '209707_at', '227158_at', '236075_s_at', '218664_at', '220966_x_at', '217772_s_at', '238032_at', '209902_at', '203647_s_at', '201722_s_at', '212296_at', '205204_at', '217835_x_at', '219446_at', '215983_s_at', '205452_at', '219374_s_at', '209398_at', '235198_at

Now we need to convert the probe IDs into entrez IDs for the comparisons. Let's use the GPL570 from the fchl GSE we loaded earlier.

In [32]:
gpl570 = fchl.gpls['GPL570'].table
gpl570.index = gpl570['ID']
# Nesting a list comprehension in a dictionary comprehension
modules_entrez = {i:set([int(i3) for i2 in [i.split(' /// ') for i in list(gpl570.loc[modules[i],'ENTREZ_GENE_ID'].dropna())] for i3 in i2]) for i in modules}
print(list(modules_entrez['Tan'])[0:100])

[100653057, 8195, 151556, 2055, 124936, 389136, 55313, 51222, 10272, 8228, 6185, 127018, 4139, 8239, 51, 139322, 57403, 10302, 4162, 348235, 79947, 57419, 10318, 4176, 114780, 4190, 98, 51304, 114793, 108, 8301, 112752, 51313, 125058, 2182, 6282, 2195, 2202, 10400, 84129, 84134, 51367, 2230, 2237, 10440, 208, 2258, 211, 57569, 114915, 101927144, 8436, 10486, 10491, 10492, 8459, 162073, 2331, 284, 28962, 22821, 90410, 28971, 301, 308, 8501, 84279, 51524, 51527, 51528, 22859, 84305, 90459, 8540, 8544, 6502, 55661, 368, 8560, 22900, 51573, 10614, 55681, 10626, 10627, 22919, 29071, 55701, 27031, 27032, 10651, 412, 51619, 373156, 8613, 8611, 427, 6584, 80315, 444]


### Compare all modules vs. FCHL DEGs
All the module data is loaded up so lets do comparison number 4, which is module gene membership vs. FCHL DEGs. Because there are multiple WGCNA modules we need to loop the analysis. Which also means we need to initialize a data structure to hold our results. Let's use a pandas DataFrame with the rows being the comparisons, and the columns being k, M, n, N, and P-value.

In [34]:
## 
module_vs_FCHL = pd.DataFrame(index=modules_entrez.keys(),columns=['k','M','n','N','P-value'])
for module in modules_entrez:
    k = len(FCHL_DEGs.intersection(modules_entrez[module]))
    M = len(allGenes)
    n = len(FCHL_DEGs)
    N = len(modules_entrez[module])
    p_value = hypergeom.sf(k, M, n, N)
    module_vs_FCHL.loc[module] = [k,M,n,N,p_value]

In [35]:
module_vs_FCHL.shape

(29, 5)

We just did 29 independent hypergeometric enrichment tests. We need to account for those tests using multiple hypothesis correction. We will use the statsmodels.stats.multitest.multipletests to correct the p-values and then add that to the DataFrame as the column 'Adj. P-value'.

In [37]:
module_vs_FCHL['Adj. P-value'] = multitest.multipletests(list(module_vs_FCHL['P-value']))[1]

C:\Users\cplaisie\AppData\Local\anaconda3\Lib\site-packages\statsmodels\stats\multitest.py:186: RuntimeWarning: divide by zero encountered in log1p
  np.log1p(-pvals))


In [38]:
# Print result
print(module_vs_FCHL.sort_values('Adj. P-value'))

                 k     M     n     N   P-value  Adj. P-value
Black          259  9946  2058   485       0.0  2.569694e-59
Magenta        289  9946  2058   573       0.0  2.658073e-59
Midnightblue   130  9946  2058   246       0.0  9.765853e-29
Greenyellow    117  9946  2058   302       0.0  2.380007e-12
Purple         184  9946  2058   581       0.0  1.315557e-09
Tan            137  9946  2058   431       0.0  2.779366e-07
Skyblue         50  9946  2058   122       0.0  1.958047e-06
White           47  9946  2058   123  0.000002  4.931629e-05
Darkgreen       64  9946  2058   185  0.000003  6.537829e-05
Green          222  9946  2058   862  0.000074  1.480296e-03
Blue           269  9946  2058  1134  0.003683  6.770022e-02
Darkgrey        40  9946  2058   152  0.036933  4.920523e-01
Lightcyan       66  9946  2058   278  0.090465  7.986864e-01
Orange          34  9946  2058   136  0.089979  7.986864e-01
Red            148  9946  2058   656  0.102427  8.022817e-01
Lightyellow     46  9946

Now we will write out enrichment analysis as a CSV.

In [40]:
# Write out CSV file
module_vs_FCHL.sort_values('Adj. P-value').to_csv('modules_vs_FCHL_DEGs.csv')

### Compare all modules vs. rs3737787 correlated/upregulated genes (alpha <= 0.01)
All the module data is loaded up so lets do comparison number 5, which is module gene membership vs. rs3737787 correlated/upregulated genes (alpha <= 0.01). Again, because there are multiple WGCNA modules we need to loop the analysis. Which also means we need to initialize a data structure to hold our results. Let's use a pandas DataFrame with the rows being the comparisons, and the columns being k, M, n, N, and P-value.

In [42]:
module_vs_rs3737787 = pd.DataFrame(index=modules_entrez.keys(),columns=['k','M','n','N','P-value'])
for module in modules_entrez:
    k = len(rs3737787_corr_up_0_01.intersection(modules_entrez[module]))
    M = len(allGenes)
    n = len(rs3737787_corr_up_0_01)
    N = len(modules_entrez[module])
    p_value = hypergeom.sf(k, M, n, N)
    module_vs_rs3737787.loc[module] = [k,M,n,N,p_value]

In [43]:
module_vs_rs3737787.shape

(29, 5)

We just did 29 independent hypergeometric enrichment tests. We need to account for those tests using multiple hypothesis correction. We will use the statsmodels.stats.multitest.multipletests to correct the p-values and then add that to the DataFrame as the column 'Adj. P-value'.

In [45]:
# Add multiply hypothesis correction
module_vs_rs3737787['Adj. P-value'] = multitest.multipletests(list(module_vs_rs3737787['P-value']))[1]

In [46]:
# Print result
print(module_vs_rs3737787.sort_values('Adj. P-value'))

                k     M    n     N   P-value  Adj. P-value
Tan            48  9946  145   431       0.0  9.941453e-30
Turquoise      62  9946  145  1369       0.0  3.992076e-17
Blue           36  9946  145  1134  0.000002  4.058502e-05
Darkturquoise   7  9946  145   171   0.00351  8.735700e-02
Yellow         17  9946  145  1019  0.227713  9.984352e-01
Darkorange      2  9946  145   155  0.394268  9.999940e-01
Skyblue         1  9946  145   122  0.533912  1.000000e+00
Black           6  9946  145   485  0.565862  1.000000e+00
Orange          1  9946  145   136  0.593096  1.000000e+00
Darkgrey        1  9946  145   152  0.653711  1.000000e+00
Darkred         1  9946  145   195  0.781355  1.000000e+00
White           0  9946  145   123  0.837591  1.000000e+00
Red             6  9946  145   656  0.850755  1.000000e+00
Magenta         0  9946  145   573  0.999828  1.000000e+00
Grey            3  9946  145   493  0.933745  1.000000e+00
Lightcyan       1  9946  145   278  0.916775  1.000000e+

We just did 29 independent hypergeometric enrichment tests. We need to account for those tests using multiple hypothesis correction. We will use the statsmodels.stats.multitest.multipletests to correct the p-values and then add that to the DataFrame as the column 'Adj. P-value'.

In [48]:
module_vs_rs3737787.sort_values('Adj. P-value').to_csv('modules_vs_rs3737787_corr_up_0_01.csv')

#### Interpretation
The Tan module is pretty amazing in that it is associated with both FCHL and rs3737787. We should look into that module deeper.

### Functional enrichment analysis with the Tan module
With functional enrichment analysis we can attribute biological functions to the Tan module. First, we need to convert the Entrez IDs into gene symbols using the [mygene](https://pypi.org/project/mygene/) package, and then compute the functional enrichment using [enrichr](https://maayanlab.cloud/Enrichr/) in [gseapy](https://gseapy.readthedocs.io/en/latest/introduction.html).

First let's convert the tan module:

In [50]:
# Map entrez gene IDs to gene symbols
tan_mapped = mg.querymany(list(modules_entrez['Tan']), scopes='entrezgene', species=9606, as_dataframe=True)
tan_symbols = list(tan_mapped['symbol'].dropna())

10 input query terms found no hit:	['100653057', '101927144', '102724112', '101929792', '100506538', '101930322', '102724985', '1019304


Next let's also convert the background:

In [52]:
background_mapped = mg.querymany(list(allGenes), scopes='entrezgene', species=9606, as_dataframe=True)
background_symbols = list(background_mapped['symbol'].dropna())

123 input query terms found no hit:	['100500840', '100861437', '100272216', '101060275', '101060386', '101060399', '101060405', '1010605


What are the possible databses for functional enrichment analysis?

In [54]:
names = gp.get_library_name()
print(names)

['ARCHS4_Cell-lines', 'ARCHS4_IDG_Coexp', 'ARCHS4_Kinases_Coexp', 'ARCHS4_TFs_Coexp', 'ARCHS4_Tissues', 'Achilles_fitness_decrease', 'Achilles_fitness_increase', 'Aging_Perturbations_from_GEO_down', 'Aging_Perturbations_from_GEO_up', 'Allen_Brain_Atlas_10x_scRNA_2021', 'Allen_Brain_Atlas_down', 'Allen_Brain_Atlas_up', 'Azimuth_2023', 'Azimuth_Cell_Types_2021', 'BioCarta_2013', 'BioCarta_2015', 'BioCarta_2016', 'BioPlanet_2019', 'BioPlex_2017', 'CCLE_Proteomics_2020', 'CORUM', 'COVID-19_Related_Gene_Sets', 'COVID-19_Related_Gene_Sets_2021', 'Cancer_Cell_Line_Encyclopedia', 'CellMarker_2024', 'CellMarker_Augmented_2021', 'ChEA_2013', 'ChEA_2015', 'ChEA_2016', 'ChEA_2022', 'Chromosome_Location', 'Chromosome_Location_hg19', 'ClinVar_2019', 'DGIdb_Drug_Targets_2024', 'DSigDB', 'Data_Acquisition_Method_Most_Popular_Genes', 'DepMap_CRISPR_GeneDependency_CellLines_2023', 'DepMap_WG_CRISPR_Screens_Broad_CellLines_2019', 'DepMap_WG_CRISPR_Screens_Sanger_CellLines_2019', 'Descartes_Cell_Types_and

Now let's conduct the functional enrichment analysis for 'GO_Biological_Process_2023' and 'KEGG_2021_Human':

In [56]:
enr1 = gp.enrichr(gene_list=tan_symbols, gene_sets=['GO_Biological_Process_2023','KEGG_2021_Human'], organism='Human', outdir='Tan_functional_enrichment', cutoff=0.05, no_plot=True) #, background=background_symbols) # tan_symbols
print(enr1.results)

                        Gene_set  \
0     GO_Biological_Process_2023   
1     GO_Biological_Process_2023   
2     GO_Biological_Process_2023   
3     GO_Biological_Process_2023   
4     GO_Biological_Process_2023   
...                          ...   
2318             KEGG_2021_Human   
2319             KEGG_2021_Human   
2320             KEGG_2021_Human   
2321             KEGG_2021_Human   
2322             KEGG_2021_Human   

                                                   Term Overlap   P-value  \
0     Positive Regulation Of Phosphoprotein Phosphat...    5/18  0.000028   
1     Positive Regulation Of Phosphatase Activity (G...    5/22  0.000079   
2                 Lipoprotein Localization (GO:0044872)    4/12  0.000084   
3     Regulation Of Cell Communication By Electrical...    4/12  0.000084   
4     Dolichol-Linked Oligosaccharide Biosynthetic P...    4/15  0.000220   
...                                                 ...     ...       ...   
2318                        

Please look into the 'Tan_functional_enrichment' directory which should have three files:
- GO_Biological_Process_2021.Tan_module.enrichr.reports.txt - GO BP enrichment
- gseapy.enrichr.Tan_module.log - log file for analysis
- KEGG_2021_Human.Tan_module.enrichr.reports.txt - KEGG enrichment

The header for the enrichment anlaysis contains:

`Gene_set	Term	Overlap	P-value	Adjusted P-value	Old P-value	Old Adjusted P-value	Odds Ratio	Combined Score	Genes`